# Descarga de 1 archivo de audio maestro para aprendizaje - *.wav

In [ ]:
#Para descargar videos de Youtube
!pip install pytube

#Para procesamiento de audio, convertir formatos de archivos de audio en otros formatos, convertir tasas de muestreo, para aplicar efectos de sonido, reproducir y grabar archivos de audio.
!apt -qq install -y sox
!apt -qq install -y sox libsox-fmt-mp3
!pip install sox
!git clone https://github.com/rabitt/pysox.git
!cd /content/pysox
!python /content/pysox/setup.py install
!pip install git+https://github.com/rabitt/pysox.git

#convertidor de audio y video. Se usa para convertir WAV a MP3 y viceversa.
!pip3 install ffmpeg
!apt -qq install -y ffmpeg

#Para reproducir, dividir, integrar o editar los archivos de audio únicamente con extensión .wav
!pip3 install pydub

#Para extraer datos de una hoja de cálculo, solo para archivos de extensión .xls
!pip3 install xlrd
!pip3 install --upgrade xlrd

#Para manejar y analizar estructuras de datos.
!pip3 install pandas
!pip3 install --upgrade pandas


In [ ]:
import os
import subprocess
import librosa
import librosa.display
import sox
import pandas
import string
import traceback
import sys
from pytube import YouTube
from pathlib import Path
from IPython.display import Audio
from pydub import AudioSegment
from pydub.utils import make_chunks
from google.colab import drive
drive.mount('/content/drive/')


In [ ]:
# Constantes y Variables de sesión

#C_MAIN_AUDIO_PATH = '/home/jmmiguez/proyectoAudio/audio/'
C_MAIN_AUDIO_PATH = '/content/drive/MyDrive/proyectoAudio/audio/'

#C_DATASET_AUDIO_PATH = "/home/jmmiguez/proyectoAudio/audio/Downloads/"
C_DATASET_AUDIO_PATH = "/content/drive/MyDrive/proyectoAudio/audio/Downloads/"

V_NOMBRE_ARCHIVO_MAESTRO = 'audio_maestro' 
V_PATH_Y_NOMRE_ARCHIVO_MP4 = C_DATASET_AUDIO_PATH + V_NOMBRE_ARCHIVO_MAESTRO + '.mp4'
V_PATH_Y_NOMRE_ARCHIVO_WAV = C_DATASET_AUDIO_PATH + V_NOMBRE_ARCHIVO_MAESTRO + '.wav'

#dirección del video de descarga en YouTube
V_VIDEO_LINK = 'https://youtu.be/aPp5lcqgISk'

#nombre del archivo excel
V_LABELS_DESDE_EXCEL = 'DonaldTrumpAI.xls'

#ruta y directorio con el archivo excel
V_PATH_Y_NOMBRE_ARCHIVO_XLS = C_MAIN_AUDIO_PATH + V_LABELS_DESDE_EXCEL
#V_CANT_LABELS_EXCEL = 1
V_CHUNK_LENGHT = 1000


In [ ]:
# Funciones para descargar audio y convertirlo a WAV

def cargarAudio():
    print('2.0. Cargar audio')
    try:
        if os.path.isfile(C_DATASET_AUDIO_PATH + V_NOMBRE_ARCHIVO_MAESTRO + '.wav') is True:
            V_COMMAND = "rm " + C_DATASET_AUDIO_PATH + V_NOMBRE_ARCHIVO_MAESTRO + '.wav'
            print('2.1. command >>',V_COMMAND)
            subprocess.call(V_COMMAND, shell=True)
            print('2.2. Archivo mp4 eliminado')

        if not(os.path.exists(C_DATASET_AUDIO_PATH + V_NOMBRE_ARCHIVO_MAESTRO + '.wav')):
          V_VIDEO = YouTube(V_VIDEO_LINK)        
          V_AUDIO = V_VIDEO.streams.filter(only_audio=True).first()
          if V_AUDIO.subtype == 'mp4':
              V_AUDIO.download(C_DATASET_AUDIO_PATH, filename=V_PATH_Y_NOMRE_ARCHIVO_MP4, filename_prefix='')
          print('2.3. Descargó el archivo: "{}{}"'.format(C_DATASET_AUDIO_PATH, V_NOMBRE_ARCHIVO_MAESTRO+'.'+V_AUDIO.subtype))
    except:
        print("Error: cargarAudio")
        traceback.print_exc()
    return None
  
def convertirMP4aWAV():
    print('3.0. Convertir el archivo mp4 a wav')
    try:
        if not(os.path.exists(V_PATH_Y_NOMRE_ARCHIVO_WAV)):
          V_COMMAND = "ffmpeg -i" + " " + V_PATH_Y_NOMRE_ARCHIVO_MP4 + " " + "-ar 16000 -vn" + " " + V_PATH_Y_NOMRE_ARCHIVO_WAV
          print('3.1. $ ',V_COMMAND)
          subprocess.call(V_COMMAND, shell=True)
          print('3.2. archivo convertido de mp4 a wav')
    except:
        print("Error: convertirMP4aWAV")
        traceback.print_exc()
    return None

def identificarArchivoAudioSinPreProcesar():
    print('4.0. Identificar los archivos de audio sin pre-procesar')
    try:
        V_AUDIO_PATHS = []
        for subdir in os.listdir(C_DATASET_AUDIO_PATH):
            V_SUBDIR_PATH = Path(C_DATASET_AUDIO_PATH) / subdir
            V_LAST_PART = V_SUBDIR_PATH.parts[-1]
            if V_LAST_PART.endswith(".wav"):
                print('4.1. {}'.format(V_SUBDIR_PATH))
                V_AUDIO_PATHS += [V_SUBDIR_PATH]
        print("4.2. Encontró {} archivo/s, en {} directorios".format(len(V_AUDIO_PATHS), len(os.listdir(C_DATASET_AUDIO_PATH))))
        for V_INDEX in (V_AUDIO_PATHS):
            print('4.3. Archivo encontrado V_INDEX = {}'.format(V_INDEX))

    except:
        print("Error: identificarArchivoAudioSinPreProcesar")
        traceback.print_exc()
    return V_INDEX

def eliminarArchivoMP4():
    try:
        if os.path.isfile(V_PATH_Y_NOMRE_ARCHIVO_MP4) is True:
            command = ("rm " + "{}".format(V_PATH_Y_NOMRE_ARCHIVO_MP4))
            print('5.0. command >>',command)
            subprocess.call(command, shell=True)
            print('5.1. Archivo mp4 eliminado')
    except:
        print("Error: eliminarArchivoMP4")
        traceback.print_exc()
    return None


In [ ]:
#Descargar audio y convertirlo a *.wav
try:
    cargarAudio()
    convertirMP4aWAV()
    V_INDEX = identificarArchivoAudioSinPreProcesar()
    eliminarArchivoMP4()
    print('5.2. Fin correcto - Parte 1')
except:
    print("Error: Nombre del hablante")
    traceback.print_exc()


In [ ]:
df = pandas.read_excel(V_PATH_Y_NOMBRE_ARCHIVO_XLS)
count = df.shape[0]

for index, row in df.iterrows():
    print("segs_ini={} segs_fin={} nombre={}".format(row[0], row[1], row[2]))
    if index == count - 1:
        break


# División de archivo maestro en múltiples muestras normalizadas

In [ ]:
#Funciones para normalizar Audio

def normalizarAudioCrudo(V_SEGS_DESDE, V_SEGS_HASTA):
    try:
        print('7.0. Normalizar el archivo crudo, usando SOX')
        V_TFM = sox.Transformer()
        V_TFM.rate(samplerate=16000)
        V_TFM.norm(db_level=-3)
        print('7.1. Recortar el audio maestro entre segundos de inicio y fin')
        V_TFM.trim(V_SEGS_DESDE, V_SEGS_HASTA)
        V_TFM.silence(
                    location = 0,
                    silence_threshold = 0.5,
                    min_silence_duration = 0.1,
                    buffer_around_silence = False
                   )
        print('7.2. Definir el nombre del archivo de salida normalizado')
        V_PATH_Y_NOMRE_ARCHIVO_NORMALIZADO_WAV = "{}{}_normal.wav".format(C_DATASET_AUDIO_PATH,V_NOMBRE_ARCHIVO_MAESTRO)
        V_TFM.build("{}".format(V_INDEX), "{}".format(V_PATH_Y_NOMRE_ARCHIVO_NORMALIZADO_WAV))
        V_TFM.effects_log
    except:
        print("Error: normalizarAudioCrudo")
        traceback.print_exc()
    return V_PATH_Y_NOMRE_ARCHIVO_NORMALIZADO_WAV

def dividirArchivoNormalizadoEnChunks1seg(V_PATH_Y_NOMRE_ARCHIVO_NORMALIZADO_WAV, V_NOMBRE_HABLANTE):
    try:
        print('8.0. Dividir el archivo normalizado, en chunks de 1 segundo.')
        myaudio = AudioSegment.from_file("{}".format(V_PATH_Y_NOMRE_ARCHIVO_NORMALIZADO_WAV) , "wav") 
        chunks = make_chunks(myaudio, V_CHUNK_LENGHT) 

        for i, chunk in enumerate(chunks):
            V_PATH_Y_NOMRE_ARCHIVO_WAV = C_DATASET_AUDIO_PATH + V_NOMBRE_HABLANTE + '.wav'
            chunk_name = "{}".format(C_DATASET_AUDIO_PATH) + "{0}.wav".format(i+0)
            #print ("exporting {}".format(chunk_name))
            chunk.export(chunk_name, format="wav")

        command = ("rm " + "{}".format(V_PATH_Y_NOMRE_ARCHIVO_NORMALIZADO_WAV))
        #print('command >>',command)
        subprocess.call(command, shell=True)

        V_PATH_CHUNK_NORMALIZADO_WAV = C_MAIN_AUDIO_PATH + 'audio/' + V_NOMBRE_HABLANTE
        try:
            if not(os.path.exists(V_PATH_CHUNK_NORMALIZADO_WAV)):
                #print('command >> mkdir',V_PATH_CHUNK_NORMALIZADO_WAV)
                os.makedirs(V_PATH_CHUNK_NORMALIZADO_WAV)
        except:
            print("Error - al intentar crear directorio: ".format(V_PATH_CHUNK_NORMALIZADO_WAV))
            traceback.print_exc()
        try:
            command = ("mv " + "{}".format(C_DATASET_AUDIO_PATH) + "[0-9]*.wav " + V_PATH_CHUNK_NORMALIZADO_WAV)
            print('8.1. command >>',command)
            subprocess.call(command, shell=True)
        except:
            print("Error - al mover chunk {}".format((C_DATASET_AUDIO_PATH) + "[0-9]*.wav " + V_PATH_CHUNK_NORMALIZADO_WAV))
            traceback.print_exc()
    except:
        print("Error: dividirArchivoNormalizadoEnChunks1seg")
        traceback.print_exc()
    return None

def eliminarChunks(V_PATH):
    try:
        os.chdir(V_PATH) 
        print("removiendo chunks de {}",format(V_PATH))
        for f in os.listdir('.'):
            if any(x in f for x in string.digits) and f.endswith('.wav'):
              try:
                os.remove(f)
              except:
                print("Error - al eliminar chunks")
        print("chunks removidos")
    except:
        print("Error - al eliminar chunks {}".format((V_PATH) + "[0-9]*.wav "))
        traceback.print_exc()
    return None


In [ ]:
# Normalización y división en múltiples muestras

df = pandas.read_excel(V_PATH_Y_NOMBRE_ARCHIVO_XLS)
count = df.shape[0]

for index, row in df.iterrows():
    print("segs_ini={} segs_fin={} nombre={}".format(row[0], row[1], row[2]))
    V_SEGS_DESDE=row[0]
    V_SEGS_HASTA=row[1]
    V_NOMBRE_HABLANTE=row[2]
    V_PATH_Y_NOMRE_ARCHIVO_NORMALIZADO_WAV = normalizarAudioCrudo(V_SEGS_DESDE, V_SEGS_HASTA)
    eliminarChunks(C_MAIN_AUDIO_PATH + 'audio/' + V_NOMBRE_HABLANTE)
    dividirArchivoNormalizadoEnChunks1seg(V_PATH_Y_NOMRE_ARCHIVO_NORMALIZADO_WAV, V_NOMBRE_HABLANTE)
    if index == count - 1:
        break
eliminarChunks(C_DATASET_AUDIO_PATH)    
print('Fin correcto - Parte 2')
